In [34]:
import numpy as np

EPS=1e-06

def bisection(f, l, r, eps=EPS):
    while r - l > eps:
        m = (l + r) / 2
        if f(m) * f(l) <= 0:
            r = m
        else:
            l = m
    return l

def newton(f, x0, iters=20):
    sequence = [x0]
    for _ in range(iters):
        sequence.append(f(sequence[-1]))
    return sequence[-1]

def trisection(f, l, r, iters=20):
    for _ in range(iters):
        a = (2 * l + r) / 3
        b = (l + 2 * r) / 3
        if f(a) < f(b):
            r = b
        else:
            l = a
    return l

In [35]:
# задание 1.1
def nth_root1(n, a):
    assert not (n % 2 == 0 and a < 0), 'Negative number %d doesn\'t have real %s root' \
    % (a, ('square' if n == 2 else ("%dth" % n)))
    
    def f(x):
        return x ** n - abs(a)
    
    root = bisection(f, 0, max(1, abs(a)))
    return -root if a < 0 else root

In [57]:
assert abs(nth_root1(2, 2) - np.sqrt(2)) < EPS
assert abs(nth_root1(3, -27) + 3) < EPS

Задание 1.2 

Рассмотрим следующую последовательность: 
$x_{k+1}=\frac{1}{n}\left((n-1)\cdot x_k+\frac{a}{x_k^{n-1}}\right)$ 

1. $\left.\frac{d}{dx}\left((n-1)\cdot x_k+\frac{a}{x_k^{n-1}}\right)\right|_{x=\sqrt[n]{a}}=(n-1)\left.\left(1-\frac{a}{x^n}\right)\right|_{x=\sqrt[n]{a}}=0$

2. Пусть $a > 0$ (для отрицательных $a$ можем вынести минус из под корня). $x_k$ возрастает, если $x_0\in(0, \sqrt[n]{a})$, и убывает, если $x_0>\sqrt[n]{a}$. То есть последовательность $x_k$ сходится, причем сходится к корню уравнения $x^n - a = 0$, то есть к $\sqrt[n]{a}$. Так как $f'$ непрерывна на $(0, +\infty)$, то она непрерывна по Липшицу на любом замкнутом отрезке $[a; b]\subset(0, \infty)$.

Таким образм, необходимые условия выполняются, и с некоторого момента метод Ньютона имеет квадратичную сходимость.

In [37]:
# задание 1.2
def nth_root2(n, a):
    assert not (n % 2 == 0 and a < 0), 'Negative number %d doesn\'t have real %s root' \
    % (a, ('square' if n == 2 else ("%dth" % n)))
    
    def f(x):
        return ((n - 1) * x + a / x ** (n - 1)) / n
    
    return newton(f, a)

In [56]:
assert abs(nth_root2(2, 2) - np.sqrt(2)) < EPS
assert abs(nth_root2(3, -27) + 3) < EPS

Задание 2.1

Для поиска нужного разбиения нам нужно найти точки, в которых многочлен меняет направление роста, то есть локальные максимумы и минимумы, а для этого найти нули производной многочлена и выбрать из них те, которые являются локальными экстремумами. Для того чтобы найти нули производной, можно найти искомое разбиение для производной, а затем на каждом отрезке найти корень производной с помощью бисекции.

In [39]:
# задание 2.1
def poly(coefs):
    n = len(coefs) - 1
    return lambda x: np.sum(np.array([x ** (n - i) for i in range(n + 1)]) * coefs)

def dpoly_coefs(coefs):
    n = len(coefs) - 1
    return [(n - i) * coef for i, coef in enumerate(coefs[:-1])]

def rec_partition(coefs, l0, r0):
    # избавляемся от старших коэффициентов равных нулю
    while len(coefs) > 0 and coefs[0] == 0:
        coefs = coefs[1:]
        
    if len(coefs) < 2:
        return None

    dcoefs = dpoly_coefs(coefs)
    f = poly(coefs)
    df = poly(dcoefs)
    
    if len(coefs) == 2:
        return None if f(l0) * f(r0) >= 0 else [[l0, r0]]
    
    dpartition = rec_partition(dcoefs, l0, r0)
    if dpartition is None:
        return None
    droots = [bisection(df, l, r) for l, r in dpartition]
    
    partition = [[l0, r0]]
    for i, droot in enumerate(droots):
        l, r = partition.pop(-1)
        if abs(f(droot)) > EPS and f(l) * f(droot) < 0:
            partition += [[l, droot], [droot, r]]
        else:
            partition.append([l, r])
    
    # проверяем, что последний добавленный отрезок удовлетворяет нужному условию
    if len(partition) > 1:
        l, r = partition.pop(-1)
        if f(l) * f(r) >= 0:
            partition[-1][1] = r
        else:
            partition.append([l, r])
    
    return partition  

# возвращает None, когда требуемое разбиение не существует
def get_partition(coefs, l0, r0):
    f = poly(coefs)
    partition = rec_partition(coefs, l0, r0)
    if len(partition) == 1 and f(l0) * f(r0) >= 0:
        return None
    return partition  

In [40]:
# задание 2.2
def find_roots(coefs, l0, r0):
    partition = get_partition(coefs, l0, r0)
    f = poly(coefs)
    return [bisection(f, l, r) for l, r in partition]

In [41]:
a = np.array(find_roots([1, 0, -5, 0, 4], -3, 3)) - [-2, -1, 1, 2]
assert np.all(abs(a) < EPS)
b =  np.array(find_roots([1, 0, -5, 0, 4, 0], -3, 3)) - [-2, -1, 0, 1, 2]
assert np.all(abs(b) < EPS)
c =  np.array(find_roots([1, -2, -5, 10, 4, -8], -3, 3)) - [-2, -1, 1] 
assert np.all(abs(c) < EPS)

Задание 2.3

Точка глобального минимума на отрезке является либо нулём производной, либо концом отрезка.

In [42]:
# задание 2.3
def find_abs_min(coefs, l, r):
    f = poly(coefs)
    dcoefs = dpoly_coefs(coefs)
    droots = find_roots(dcoefs, l, r)
    return min(map(lambda x: f(x), droots + [l, r]))

Задание 3

$f(x) = e^{ax} + e^{-bx} + c\cdot(x-d)^2$

$f'(x) = a\cdot e^{ax} - b\cdot e^{-bx} + 2c\cdot(x-d)$

$f''(x) = a^2\cdot e^{ax} + b^2\cdot e^{-bx} + 2c$

$f''(x) > 0$ на $\mathbb{R}$, то есть $f'(x)$ строго возрастает, при этом на $-\infty$ $f'(x)$ стремится к $-\infty$, на $\infty$ стремится к $\infty$, и значит уравнение $f'(x)=0$ имеет единственный корень, который также является точкой минимума $f(x)$.

In [43]:
def f(a, b, c, d):
    return lambda x: np.e ** (a * x) + np.e ** (-b * x) + c * (x - d) ** 2

def df(a, b, c, d):
    return lambda x: a * np.e ** (a * x) - b * np.e ** (-b * x) + 2 * c * (x - d)

def ddf(a, b, c, d):
    return lambda x: a ** 2 * np.e ** (a * x) + b ** 2 * np.e ** (-b * x) + 2 * c

In [44]:
# задание 3.1
def get_minimum1(a, b, c, d):
    l0 = min(d, -a / b / (a + b))
    r0 = max(d, b / a / (a + b))
    x_min = bisection(df(a, b, c, d), l0, r0)
    return f(a, b, c, d)(x_min)

In [55]:
assert abs(get_minimum1(1, 2, 3, -4) - 34.5867) < 1e-04

In [46]:
# задание 3.2
def get_minimum2(a, b, c, d):
    x_min = newton(lambda x: x - df(a, b, c, d)(x) / ddf(a, b, c, d)(x), 0)
    return f(a, b, c, d)(x_min)

In [53]:
assert abs(get_minimum2(1, 2, 3, -4) - 34.5867) < 1e-04

In [50]:
# задание 3.3
def get_minimum3(a, b, c, d):
    l0 = min(d, -a / b / (a + b))
    r0 = max(d, b / a / (a + b))
    x_min = trisection(f(a, b, c, d), l0, r0)
    return f(a, b, c, d)(x_min)

In [54]:
assert abs(get_minimum3(1, 2, 3, -4) - 34.5867) < 1e-04